``` SQL
UPDATE ALIGNEMENTS
SET pointsActuels = (SELECT COUNT(Event) AS NbPTS     
  FROM PBP
       INNER JOIN
       ALIGNEMENTS ON (PBP.p1_ID = ALIGNEMENTS.idNHL) OR 
                      (PBP.p2_ID = ALIGNEMENTS.idNHL) OR 
                      (PBP.p3_ID = ALIGNEMENTS.idNHL) 
 WHERE EVENT = "GOAL" AND 
       statutJoueur = "Alignement" AND 
       PBP.Date > ALIGNEMENTS.dateDebut AND
       PBP.Date <= ALIGNEMENTS.dateFin
GROUP BY idAlignement)
 ```

Une cellule qui:

    - (DONE) (majPoints) qui met à jour les buts, passes et PJ des joueurs selon la période donnée dans Alignements
    - (DONE) qui met à jour le nombre de points de chaque pooler
    - qui met à jour la masse salariale de chaque pooler
    - (majJson) qui met à jour les json
    - (testPierre) exécute le sync avec GitHub

In [10]:
#variables globales

conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
fin = " WHERE EVENT = 'GOAL' AND PBP.Date > ALIGNEMENTS.dateDebut AND PBP.Date <= ALIGNEMENTS.dateFin AND PBP.Date > '2019-08-25' GROUP BY idAlignement)) WHERE idAlignement = ALIGNEMENTS.idAlignement)"

#fonctions
def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

def writeUpdateDebut(typeScore):
    updateDebut = "UPDATE ALIGNEMENTS SET " + typeScore + "Actuels = (SELECT Nb" + typeScore + " FROM ((SELECT idAlignement, COUNT(Event) AS Nb" + typeScore + " FROM PBP INNER JOIN ALIGNEMENTS ON"
    return updateDebut
    
def majInterne(typeScore):
    debut = writeUpdateDebut(typeScore)
    if typeScore == "buts":
        milieu = " (PBP.p1_ID = ALIGNEMENTS.idNHL)"
    elif typeScore == "assist":
        milieu = " (PBP.p2_ID = ALIGNEMENTS.idNHL) OR (PBP.p3_ID = ALIGNEMENTS.idNHL)"
    sqlUpdate = debut + milieu + fin
    execSQL(sqlUpdate, conn, write = True)

def majPointsAlignements():
    sqlUpdate = "UPDATE ALIGNEMENTS SET pointsActuels = (butsActuels + assistActuels)"
    execSQL(sqlUpdate, conn, write = True)

def majPJ():
    sqlUpdate = "UPDATE ALIGNEMENTS SET pjActuels = (SELECT NbPJ FROM (SELECT idAlignement, COUNT(DISTINCT Game_Id) AS NbPJ FROM SHIFTS INNER JOIN ALIGNEMENTS ON (SHIFTS.Player_Id = ALIGNEMENTS.idNHL) AND SHIFTS.Date > ALIGNEMENTS.dateDebut AND SHIFTS.Date <= ALIGNEMENTS.dateFin AND SHIFTS.Date > '2019-08-25' GROUP BY idAlignement) WHERE idAlignement = ALIGNEMENTS.idAlignement)"
    execSQL(sqlUpdate, conn, write = True)

def majPJBPPAlign():
    majInterne("buts")
    majInterne("assist")
    majPointsAlignements()
    majPJ()

def majScorePoolers():
    sqlUpdate = "UPDATE POOLERS SET PJ = (SELECT NbPJ FROM (SELECT idPooler, SUM(pjActuels) AS NbPJ FROM ALIGNEMENTS GROUP BY idPooler) WHERE Id = idPooler), B = (SELECT NbB FROM (SELECT idPooler, SUM(butsActuels) AS NbB FROM ALIGNEMENTS GROUP BY idPooler) WHERE Id = idPooler), A = (SELECT NbA FROM (SELECT idPooler, SUM(assistActuels) AS NbA FROM ALIGNEMENTS GROUP BY idPooler) WHERE Id = idPooler)"
    sqlUpdate2 = "UPDATE POOLERS SET Score = (POOLERS.B + POOLERS.A)"
    execSQL(sqlUpdate, conn, write = True)
    execSQL(sqlUpdate2, conn, write = True)

In [3]:
majPJBPPAlign()

In [11]:
majScorePoolers()